### Create a Langchain application
Create a new Python program and copy this code into it.

In [1]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.conversation.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import Tool
from langchain import hub
from langchain_community.tools import YouTubeSearchTool
from langchain_community.graphs import Neo4jGraph
from langchain.chains import RetrievalQA
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.chat_models import ChatOpenAI
from langchain.schema import StrOutputParser
from langchain.output_parsers.json import SimpleJsonOutputParser
from langchain.schema.messages import HumanMessage, SystemMessage
import sys
import os

# Add the parent directory of 'utilities' to the Python path
sys.path.append(r'C:\Users\Ai Sukmoren\Desktop\chatbot')

from dotenv import load_dotenv
from utilities.llms import llm

In [2]:
question = "What is Neo4j?"

response = (llm.invoke(question)).content

print(response)

Neo4j is a highly scalable, native graph database management system designed to handle complex, interconnected data. Unlike traditional relational databases that use tables and rows, Neo4j uses graph structures with nodes, edges (also called relationships), and properties to represent and store data. This model is particularly well-suited for applications that involve complex queries and relationships, such as social networks, recommendation systems, fraud detection, and network management.

Here are some key features of Neo4j:

1. **Graph Data Model**: Neo4j uses a property graph model where data is stored as nodes (entities) and relationships (edges) between those nodes. Both nodes and relationships can have properties (key-value pairs).

2. **Cypher Query Language**: Neo4j uses Cypher, a powerful and expressive graph query language designed specifically for working with graph data. Cypher queries allow for easy and intuitive traversal of the graph to find patterns and extract inform

In [ ]:
template="""
        You are a cockney fruit and vegetable seller.
        Your role is to assist your customer with their fruit and vegetable needs.
        Respond using cockney rhyming slang.

        Tell me about the following fruit: {fruit}
        Tell me about the following vegetable: {vegatable}
        """


summary_prompt_template = PromptTemplate(template=template,input_variables=["fruit","vegatable"])



response = llm_3.invoke(summary_prompt_template.format(fruit="Orange",vegatable="Cucumber"))

print(response.content)

In [11]:
template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
Tell me about the following vegatable: {vegatable}
""")

llm_chain = LLMChain(
    llm=llm,
    prompt=template,
    verbose=True
)

input_variables = {"fruit": "apple","vegatable":"cucumber"}

response = llm_chain.invoke(input_variables)

print(response['text'])



> Entering new LLMChain chain...
Prompt after formatting:

You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: apple
Tell me about the following vegatable: cucumber


> Finished chain.
Apple, mate? That's a fine bit o' scrumpy! It's the bee's knees, it is. Crunchy and juicy, like a nice bubble and squeak. Perfect for a bit of nosh or even a proper apple pie. Just make sure you don't go up the apples and pears, or you'll end up on your Jack Jones!

Now, let me tell ya about this cucumber, me old china. It's a real trouble and strife, it is. All green and firm, like a proper Jack the Ripper. You can have it in a salad or even make a smashing cucumber sandwich. Just don't forget to take the skin off, or you'll be in a right Barney Rubble!


In [ ]:
template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
Tell me about the following vegatable: {vegatable}
""")

llm_chain = (
    llm=llm,
    prompt=template,
    verbose=True
)

input_variables = {"fruit": "apple","vegatable":"cucumber"}

response = llm_chain.invoke(input_variables)

print(response['text'])

In [ ]:
template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Tell me about the following fruit: {fruit}
""")

llm_chain = LLMChain(
    llm=llm,
    prompt=template,
    output_parser=StrOutputParser()
)

response = llm_chain.invoke({"fruit": "apple"})

print(response['text'])

In [ ]:
template = PromptTemplate.from_template("""
You are a cockney fruit and vegetable seller.
Your role is to assist your customer with their fruit and vegetable needs.
Respond using cockney rhyming slang.

Output JSON as {{"description": "your response here"}}

Tell me about the following fruit: {fruit}
""")

llm_chain = LLMChain(
    llm=llm,
    prompt=template,
    output_parser=SimpleJsonOutputParser()
)

response = llm_chain.invoke({"fruit": "apple"})

print(response['text'])

## chatmodels

In [ ]:
prompt = PromptTemplate(
    template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Question: {question}
""",
    input_variables=["question"],
)

chat_chain = LLMChain(llm=llm, prompt=prompt)

response = chat_chain.invoke({"question": "What is the weather like?"})

print(response)

In [ ]:
current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Polzeath", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""

prompt = PromptTemplate(
    template="""You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Context: {context}
Question: {question}
""",
    input_variables=["context", "question"],
)

chain = LLMChain(llm=llm, prompt=prompt)

response = chain.invoke(
    {
        "context": current_weather,
        "question": "What is the weather like?"
     
    }
)
print(response['text'])

### Chat_history

In [7]:
prompt = PromptTemplate(
    template="""
You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: {chat_history}
Context: {context}
Question: {question}
""",
    input_variables=["chat_history", "context", "question"],
)

memory = ConversationBufferMemory(
    memory_key="chat_history", input_key="question", return_messages=True
)

chat_chain = LLMChain(llm=llm, prompt=prompt, memory=memory, verbose=True)

current_weather = """
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }"""
    
while True:
    question = input("> ")
    if question == 'exit':
        break
    else:
        response = chat_chain.invoke({"context": current_weather, "question": question})
        print(response["text"])



> Entering new LLMChain chain...
Prompt after formatting:

You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: []
Context: 
    {
        "surf": [
            {"beach": "Fistral", "conditions": "6ft waves and offshore winds"},
            {"beach": "Bells", "conditions": "Flat and calm"},
            {"beach": "Watergate Bay", "conditions": "3ft waves and onshore winds"}
        ]
    }
Question: hi


> Finished chain.
Hey there, bro! Stoked to chat about the gnarly waves! What's the surf sitch you're curious about? 🤙


> Entering new LLMChain chain...
Prompt after formatting:

You are a surfer dude, having a conversation about the surf conditions on the beach.
Respond using surfer slang.

Chat History: [HumanMessage(content='hi'), AIMessage(content="Hey there, bro! Stoked to chat about the gnarly waves! What's the surf sitch you're curious about? 🤙")]
Context: 
    {
        "surf": [
            {"beach": 

### Agent

In [16]:
prompt = PromptTemplate(
    template="""
    You are a movie expert. You find movies from a genre or plot.

    ChatHistory:{chat_history}
    Question:{input}
    """,
    input_variables=["chat_history", "input"],
)

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

chain = LLMChain(llm=llm, prompt=prompt, memory=memory)

youtube = YouTubeSearchTool()

tools = [
    Tool.from_function(
        name="Movie Chat",
        description="For when you need to chat about movies. The question will be a string. Return a string.",
        func=chain.run,
        return_direct=True
    ),
    Tool.from_function(
        name="Movie Trailer Search",
        description="Use when needing to find a movie trailer. The question will include the word 'trailer'. Return a link to a YouTube video.",
        func=youtube.run,
        return_direct=True
    )
]
agent_prompt = hub.pull("hwchase17/react-chat")

agent = create_react_agent(llm, tools, agent_prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    max_interations=3,
    verbose=True,
    handle_parse_errors=True
    )

while True:
    question = input("> ")
    if question == 'exit':
        break
    else:
        response = agent_executor.invoke({"input": question})
        print(response['output'])



> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: Hello! How can I assist you today?
```

> Finished chain.
{'input': 'hi', 'chat_history': [HumanMessage(content='hi'), AIMessage(content='Hello! How can I assist you today?\n```')], 'output': 'Hello! How can I assist you today?\n```'}


> Entering new AgentExecutor chain...
```
Thought: Do I need to use a tool? No
Final Answer: Determining the "best" horror movie of 2018 can be subjective as it depends on individual tastes and criteria for what makes a horror movie great. However, one film that received critical acclaim and was popular among audiences is "Hereditary" directed by Ari Aster. It's known for its intense atmosphere, strong performances, particularly by Toni Collette, and its chilling and unsettling narrative. Other notable horror films from 2018 include "A Quiet Place" by John Krasinski and "Halloween" which is a sequel to the original 1978 film. Each of these films has its own un